<a href="https://colab.research.google.com/github/roman6s/SCM_Fallstudie/blob/main/Fallstudie_Version_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Framework

In [15]:
! git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

fatal: destination path 'WS24_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [16]:
! pip install -q pyscipopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 56.9 MB/s eta 0:00:00


In [17]:
import pandas as pd
from pyscipopt import Model, quicksum

## Daten laden

In [18]:
folder = "WS24_Supply_Chain_Optimierung/Daten/Fallstudie"

In [20]:
# Preisprogonose
preisprognose = pd.read_excel(f"{folder}/Preisprognosen.xlsx")


## Indexmengen

In [21]:
# Menge der Stunden des Folgetages
T = [str(i) for i in range(1, 25)]


In [22]:
# Entscheidung
D = ['buy', 'sell']

## Parameter

In [23]:
# Nominelle Speicherkapazität in MWh
cap = 40

In [24]:
# Depth of Discharge
DoD = 0.8

In [25]:
# Minimaler Stage of Charge in MWh
SOCmin = 8

In [26]:
# Maximaler Stage of Charge in MWh
SOCmax = 40

In [27]:
# C-Rate
c = 0.5

In [28]:
# Round Trip Efficiency
eta = 0.95

In [29]:
# Wirkungsgrad
mue = 0.985

In [48]:
# Zyklus
z = {t: scip.addVar(vtype="CONTINUOUS", lb=0, ub=2, name=f"z_{t}") for t in T}

In [41]:
# Anzahl der Vollzyklen
Zmax = 2

In [31]:
# MinimalPreis mit Entscheidung d zur Stunde t in €/MWh
Pmin = -500

In [32]:
# Maximalpreis mit Entscheidung d zur Stunde t in €/MWh
Pmax = 4000

In [33]:
# Zykluskosten
cost = 1500

## Optimierungsmodell

In [35]:
scip = Model()

## Entscheidungsvariablen

In [36]:
# Gebotene Menge in Stunde t für Entscheidung d
X={}
for d in D:
  for t in T:
    X[d,t] = scip.addVar(vtype="C", name=f"{d},{t}")

In [37]:
# Gebotener Preis in Stunde t für Entscheidung d
P={}
for d in D:
  for t in T:
    P[d,t] = scip.addVar(vtype="C", name=f"{d},{t}")

## Zielfunktion


Max $K = \sum_{t \in T} \left( P_{t,\text{sell}} * X_{t,\text{sell}} - P_{t,\text{buy}} * X_{t,\text{buy}} \right) - \text{cost} * \sum_{t \in T} z_t$



In [54]:
# Maximierung des Profits
scip.setObjective(
    sum(P['sell',t]*X['sell',t] for t in T) - sum(P['buy',t]*X['buy',t] for t in T) - cost*sum(z[t] for t in T),
    sense="maximize")


ValueError: SCIP does not support nonlinear objective functions. Consider using set_nonlinear_objective in the pyscipopt.recipe.nonlinear

## Entscheidungsvariablen

In [ ]:
# Beschaffungsmengen der jeweiligen Vorprodukte (Flasche und Vitaminwasser)
Z={}
for l in L:
  for i in I:
    for p in VP:
      for t in T:
        Z[l,i,p,t] = scip.addVar(vtype="C", name=f"{l},{i},{p},{t}")

In [ ]:
print('Anzahl Entscheidungsvariablen =', len(scip.getVars()))

Anzahl Entscheidungsvariablen = 360


## Parameter

In [ ]:
# Produktionskosten
pc = produktion_kosten.set_index(["Werk","Produkt"]).to_dict("dict")["Produktionskosten"] # Kosten reguläre Produktion in €/ME
oc = produktion_kosten.set_index(["Werk"]).to_dict("dict")["Kosten_pro_Zusatzkapa"] # Kosten Zusatzkapazitäten in €/Kapazitätseinheit

## Zielfunktion

### Zielfunktion
Min $K = \sum_{l,i,p \in VP,t} bc_{lp} * Z_{lipt} + \sum_{l,i,p \in VP,t} tbc_{lip} * Z_{lipt} + \sum_{i,p \in FP,t} pc_{ip} * X_{ipt} + \sum_{i,p,t} lc_{p} * B_{ipt} + \sum_{i,t} oc^+_{it} * O_{it} + \sum_{i,j,p,t} dtc_{ijp} * Y_{ijpt}$

In [ ]:
bc.get(('ErpoPlast', 'Vitaminwasser'),0)

0

In [ ]:
# Minimierung der gesamten Kosten
scip.setObjective(
    sum(bc.get((l,p),0) * Z[l,i,p,t] for l in L for i in I for p in VP for t in T) + # Kosten Beschaffung
    sum(btc.get((l,i,p),0) * Z[l,i,p,t] for l in L for i in I for p in VP for t in T) + # Transportkosten Beschaffung
    sum(pc[i,p]*X[i,p,t] for i in I for p in FP for t in T) + # Kosten der Produktion mit regulärer Kapazität
    sum(oc[i]*O[i,t] for i in I for t in T) + # Kosten der Produktion mit zusätzlicher Kapazität
    sum(lc[p]*B[i,p,t] for i in I for p in P for t in T) + # Kosten der Lagerung
    sum(dtc[i,j,p]*Y[i,j,p,t] for i in I for j in J for p in FP for t in T), # Transportkosten Distribution
    sense="minimize"
    )

## Nebenbedingungen

**(1) Lieferantenkapazitäten**

$\sum_{i} Z_{lipt} \le bcap_{lpt}$

$∀ l,p \in VP, t$

In [ ]:
for l in L:
  for p in VP:
    for t in T:
      scip.addCons(quicksum(Z[l,i,p,t] for i in I)<= bcap.get((l,p,t),0))

## Berechnung Lösung

In [ ]:
scip.optimize()
print(scip.getStatus())

optimal
